In [1]:
#Imports
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
#Define the network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Imagen con 1 canal de entrada, 6 canales de salida, 3x3 cuadrado convulucionar
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6x6 para la dimension de la imagen
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling sobre una ventana (2, 2)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # Si el tamaño es un cuadrado solo puedes especificar un numero
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    #Similar a usar la funcion flatten
    def num_flat_features(self, x):
        size = x.size()[1:] #Todas las dimensiones excepto el batch
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
#Función forward
params = list(net.parameters())
print(len(params))
print(params[0].size()) # Peso de conv1 

10
torch.Size([6, 1, 3, 3])


In [4]:
#Prueba con una entrada 32x32 aleatoria
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0616, -0.0960, -0.0445,  0.0867, -0.0185,  0.0983,  0.0206,  0.0549,
          0.0562, -0.0249]], grad_fn=<AddmmBackward>)


In [5]:
#Reset de gradientes
net.zero_grad()
out.backward(torch.randn(1, 10)) #Llamada al backward

In [6]:
#Loss function(Función de perdida)
output = net(input)
target = torch.randn(10) #dummy target, para este ejemplo
target = target.view(1, -1) 
criterion = nn.MSELoss() 

loss = criterion(output, target)
print(loss)

tensor(0.3500, grad_fn=<MseLossBackward>)


In [8]:
'''
Usando .grad_fn puedes seguir el loss en dirección contraria ( backward ) veras el grafo
convolucional así: 

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
'''
print(loss.grad_fn) #MSEloss
print(loss.grad_fn.next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #Relu

In [9]:
#Backprop del error
net.zero_grad() # borra los gradientes de todos los parametros

print('conv1.bias.grad before barckward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before barckward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 6.7811e-04,  1.2569e-03, -7.5423e-07, -7.2013e-03,  1.5710e-03,
        -6.1412e-03])


In [11]:
#Actualizar los pesos
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

#Optimizador, con la regla de actualizacion SGD
import torch.optim as optim

#Creamos el optimizador
optimizer = optim.SGD(net.parameters(), lr=0.01)

#Loop de entrenamiento
optimizer.zero_grad() # borramos los buffers de gradiante
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Actualiza